In [8]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [2]:
import yaml

In [ ]:
from pprint import pprint

In [29]:
import json

In [40]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

In [54]:
def get_fore_context(inputs):
    return inputs[: inputs.find("{end token}")].replace("{start token}","")

In [55]:
class LLM:
    def __init__(self, conf_file) -> None:
        with open(conf_file, "r", encoding="utf-8") as f:
            conf = yaml.safe_load(f.read())
        OPENAI_API_KEY = conf["OPENAI_API_KEY"]
        OPENAI_API_BASE = conf["OPENAI_API_BASE"]
        OPENAI_API_VERSION = conf["OPENAI_API_VERSION"]
        DEPLOYMENT = conf["DEPLOYMENT"]
        self.llm = AzureChatOpenAI(
            openai_api_type="azure",
            openai_api_version=OPENAI_API_VERSION,
            openai_api_base=OPENAI_API_BASE,
            openai_api_key=OPENAI_API_KEY,
            deployment_name=DEPLOYMENT,
            temperature=0,
        )

    def complete(self, query):
        """Answer to a query."""
        system_setting = SystemMessage(
            content="You help to complete the user's code.")
        
        message = HumanMessage(content=query)
        return self.llm.invoke([system_setting, message]).content
    
    def complete_code(self, code_context):
        """Take the input from the request and output.

        args:
            code_context(str): the code_context

        return(dict): the response
        """
        fore_context = get_fore_context(code_context)
        system_setting = SystemMessage(
            content="You are a code autocompleter.")
        prompt = f"""
        Please complete code for the following code. Make code completion after the end token.
        \n\n
        {code_context}
        """
        message = HumanMessage(content=prompt)
        return self.llm.invoke([system_setting, message]).content

llm = LLM(".env-35-16k.yml")

In [42]:
inputs = '{start token}import numpy as np\nimport scipy as sp\n{end token}def hello_world():\n    print("Hello world"){middle token}'
data = {"inputs": inputs, "parameters": {"max_new_tokens": 256}}

In [57]:
get_fore_context(inputs)

'import numpy as np\nimport scipy as sp\n'

In [56]:
pprint(llm.complete_code(inputs))

('Here is the completed code:\n'
 '\n'
 '```python\n'
 'import numpy as np\n'
 'import scipy as sp\n'
 '\n'
 'def hello_world():\n'
 '    print("Hello world")\n'
 '```\n'
 '\n'
 'Please let me know if you need any further assistance.')


In [6]:
false = False
generate_response = {
  "details": {
    "best_of_sequences": [
      {
        "finish_reason": "length",
        "generated_text": "test",
        "generated_tokens": 1,
        "prefill": [
          {
            "id": 0,
            "logprob": -0.34,
            "text": "test"
          }
        ],
        "seed": 42,
        "tokens": [
          {
            "id": 0,
            "logprob": -0.34,
            "special": false,
            "text": "test"
          }
        ],
        "top_tokens": [
          [
            {
              "id": 0,
              "logprob": -0.34,
              "special": false,
              "text": "test"
            }
          ]
        ]
      }
    ],
    "finish_reason": "length",
    "generated_tokens": 1,
    "prefill": [
      {
        "id": 0,
        "logprob": -0.34,
        "text": "test"
      }
    ],
    "seed": 42,
    "tokens": [
      {
        "id": 0,
        "logprob": -0.34,
        "special": false,
        "text": "test"
      }
    ],
    "top_tokens": [
      [
        {
          "id": 0,
          "logprob": -0.34,
          "special": false,
          "text": "test"
        }
      ]
    ]
  },
  "generated_text": "test"
}

In [7]:
generate_response.keys()

dict_keys(['details', 'generated_text'])